In [1]:
from common import *
from search.imports import *
from search.search import *
from search.manager import ModelManager

In [2]:
import globals as top_imports
top_imports.use_M = False
from given import *

In [3]:
F, P = pd.read_pickle(big_data/'saves/train_5fixedsince+1.32.pkl')
W, C, D = pd.read_pickle(big_data/'pair/working_2ONEorTEN.pkl'); W, C, D = O(**W), O(**C), O(**D)

### Search Params specs

In [4]:
doWork = 'solo' # <= this is the option to change!
if doWork=='solo':
#     from search.LSP_solo import LSP
    genFunc = 'gen_solo'
elif doWork=='pair':
#     from search.LSP_pair import LSP
    genFunc = 'gen_pair'
else:
    assert False

In [5]:
class LSP(O()):
    class Discrete(O()):
        stop = {} #{1:.45, 2:.5, 3:.55, 5:.6, 7:.65, 9:.7} #{1:.3, 3:.5, 5:.6, 7:.65}
        obo = {0: 0}
        enc = {
            'learning_rate': [.05],
            ('max_depth','num_leaves','num_iterations'):
                [(8,1<<8,1100),(10,1<<10,650),(-1,1<<12,450)]
        }
        
    class OneByOne(O()):
        base = .33
        class info(O()):
            a = "main [a]rray data of 3 values, [min mid max]"
            b =  "[b]ack up value, i.e. default value if array doesn't give better results"
            cast = "function to apply to values before using to cast to the right dtype"
            lim = "maximum number of iterations of searching in this hyperparameter"
        class default(O()):
            cast = keepSigFig(2)
            lim = 0
        enc = {#TODO implement 1 sided search e.g. len(a)==2
            0: {
                'min_data_in_leaf': O(a=[1,60,375], b=120, cast=round, lim=lambda lim: lim),
                'min_sum_hessian_in_leaf': O(a=[0,50,200], lim=lambda lim: lim+2),
                'lambda_l1': O(a=[0,.02,.2], b=0),
                'lambda_l2': O(a=[0,.02,.2], b=0),
            }
        }

### Features

In [6]:
fTransNewfault = ["volume","close","open","assetCodeId","assetNameId","cc","oo","acc","aoo","ccTEN","ooTEN","accTEN","aooTEN","dayOfYear","dayOfWeek","oo1Since21","oo1MaxSince21","oo1Drawdown1Since21","oo1Drawdown5Since21","oo1Drawdown10Since21","oo1MaxSince21{10}","oo1MinSince21","oo1MinSince21{10}","oo1Since62","oo1MaxSince62","oo1Drawdown1Since62","oo1Drawdown5Since62","oo1Drawdown10Since62","oo1MaxSince62{10}","oo1MinSince62","oo1MinSince62{10}","oo1Since125","oo1MaxSince125","oo1Drawdown1Since125","oo1Drawdown5Since125","oo1Drawdown10Since125","oo1MaxSince125{10}","oo1MinSince125","oo1MinSince125{10}","oo1Since250","oo1MaxSince250","oo1Drawdown1Since250","oo1Drawdown5Since250","oo1Drawdown10Since250","oo1MaxSince250{10}","oo1MinSince250","oo1MinSince250{10}","aoo1Since21","aoo1MaxSince21","aoo1Drawdown1Since21","aoo1Drawdown5Since21","aoo1Drawdown10Since21","aoo1MaxSince21{10}","aoo1MinSince21","aoo1MinSince21{10}","aoo1Since62","aoo1MaxSince62","aoo1Drawdown1Since62","aoo1Drawdown5Since62","aoo1Drawdown10Since62","aoo1MaxSince62{10}","aoo1MinSince62","aoo1MinSince62{10}","aoo1Since125","aoo1MaxSince125","aoo1Drawdown1Since125","aoo1Drawdown5Since125","aoo1Drawdown10Since125","aoo1MaxSince125{10}","aoo1MinSince125","aoo1MinSince125{10}","aoo1Since250","aoo1MaxSince250","aoo1Drawdown1Since250","aoo1Drawdown5Since250","aoo1Drawdown10Since250","aoo1MaxSince250{10}","aoo1MinSince250","aoo1MinSince250{10}","oo5","oo10","oo15","oo20","oo(10-5)","oo(15-5)","oo(20-5)","oo(15-10)","oo(20-10)","oo(20-15)","aoo5","aoo10","aoo15","aoo20","aoo(10-5)","aoo(15-5)","aoo(20-5)","aoo(15-10)","aoo(20-10)","aoo(20-15)","(it,af,it{1},aoo,acc){3/4}"]
fTransNewnewfault = fTransNewfault[:-1] + ['(aoo-acc)']
fTime = 'dayOfYear dayOfWeek'.split()
fAsset = 'assetCodeId assetNameId'.split()
fPriceAbsolute = 'close open'.split()
def select_relative(f):
    return f not in set(fTime+fAsset+fPriceAbsolute)
fInitKaggleGo = list(filter(select_relative, fTransNewnewfault)) + ['assetCodeIdWTFFlogabs', 'assetCodeIdWTFFsin2']
# _fMust = ['oo','(oo-aoo)','(cc-acc)','(oo-cc)','(aoo-acc)','vp1dd']
# fMustInclude = _fMust+[c+'1' for c in _fMust]+[c+'*' for c in _fMust]
# fMustInclude += 'af it aoo acc'.split()
class ikg(NamedO()):
    features = fInitKaggleGo
    train_on = [2009.,2009.75,2010.75,2011.5,2012.25,2012.5,2012.75,2013.,2013.25,2013.75,2014.,2014.25,2015.]
    weight = 'flat_weight'
    params = dict(early_stopping_round=50,learning_rate=.05,num_leaves=1<<12,max_depth=12,min_data_in_leaf=150,min_sum_hessian_in_leaf=50,lambda_l1=.01,lambda_l2=.01)
_ikg = ikg; del ikg
    
class bstdefs(O()):
    ikg = _ikg
    class s00(NamedO()):
        features = ["cc","oo","acc","aoo","vp10","oo5","oo10","oo15","aoo5","aoo10","aoo15","vp1dd5","vp1dd10","vp1dd15","volatility60","oo(15-10)","oo(20-15)","oo(15-5)","oo(20-10)","aoo(15-10)","aoo(20-15)","aoo(15-5)","aoo(20-10)","vp1dd(15-10)","vp1dd(20-15)","vp1dd(15-5)","vp1dd(20-10)","oo{0/1}","oo{0/2}","oo{1}","oo{2}","(oo-aoo){0/1}","(oo-aoo){0/2}","(oo-aoo){1}","(oo-aoo){2}","vp1dd{0/1}","vp1dd{0/2}","vp1dd","vp1dd{1}","vp1dd{2}","oo1Drawdown10Since62","oo1Drawdown20Since62","oo1MaxSince62{10}","oo1MaxSince62{20}","oo1MinSince62","oo1Drawup1Since62","oo1MinSince62{5}","volatility20MaxSince62","volatility20MaxSince62{5}","volatility20Drawup(10-5)Since62","volatility20Drawup(20-10)Since62","volatility20MinSince62{20}","oo1Drawdown10Since250","oo1Drawdown20Since250","oo1MaxSince250{10}","oo1MaxSince250{20}","oo1MinSince250","oo1Drawup1Since250","oo1MinSince250{5}","volatility20MaxSince250","volatility20MaxSince250{5}","volatility20Drawup(10-5)Since250","volatility20Drawup(20-10)Since250","volatility20MinSince250{20}"]
        train_on = [2010.0,2015.25,2011.5,2011.0,2012.25,2014.5,2009.5,2010.75,2011.25,2012.75,2013.0,2014.25,2015.0]
        weight = 'flat_weight'
        params = dict(learning_rate=0.05,max_depth=-1,num_leaves=1024,min_data_in_leaf=120,min_sum_hessian_in_leaf=100.0,lambda_l1=0,lambda_l2=0)

### Search Features/Samples specs

In [7]:
from features import Features
bigF = Features(F=F, P=P, W=W, C=C, D=D)
@inside(bigF, name='Pair')
class Pair(O()):
    lo = (.55, 500)
    hi = None #(.75, 9999)
    repeat = 5


class LSM(O()):
    model = O(
        time = 'time',
        value = 'y',
        target = 'target',
        weight = 'flat_weight',
    )
    
    metric = KaggleMetric()
    search = LSP
    
    t_offset = 0
    num_feats = 1
    num_samps = 1
    stop_samps = {2:.4, 3:.46, 4:.51, 5:.55, 6:.59, 7:.62, 8:float('inf')} #{1:.57, 2:.61, 3:.64, 4:float('inf')}
    
    
@inside(LSM)
class Step(metaclass=staticclass):
    _best = -float('inf')
    def func(results, *, t):
        t += 1
#         self = __class__
#         num_feats, num_samps, stop_samps = LSM.num_feats, LSM.num_samps, LSM.stop_samps

#         if results is not None and results['score'] > self._best:
#             self._best = results['score']

#         ts = t % num_samps
#         assert 0 not in stop_samps
#         if ts in stop_samps and stop_samps[ts] > self._best:
#             t = (t//num_samps + 1) * num_samps # go to next feature if met stop score criterion
#             ts = t % num_samps
            
#         if ts == 0: # if new feature, reset `self._best` score
#             self._best = -float('inf')
        
#         if t >= num_feats * num_samps:
#             return None
#         return t
        if t > 0:
            return None
        return t

@inside(LSM)
class Features(metaclass=staticclass):
    '''features selection groups'''
    def func(*, t):
#         t += LSM.t_offset
#         num_feats, num_samps = LSM.num_feats, LSM.num_samps
#         f, q = getattr(bigF, genFunc)(t // num_samps)
#         return O(Feats=f, Ctor=q)
        bigF.X, bigF.Y = F, P
        return O(Feats=bstdefs.ikg.features, Ctor='<InitKaggleGo>')

@inside(LSM)
class Samples(metaclass=staticclass):
    '''sample learning/cv split'''
    # O(method='GroupShuffleSplit.2', kwargs=dict(n_splits=<(5)many>, test_size=.5, random_state=44), groups='quarter'),
#     def ctor2samps(ctor):
#         ho = __class__.ho
#         a, b = tuple(ctor)
#         group = bigF.Y.quarter[~ho]
#         bigF._ho = bigF.X.index[ho]
#         tr, cv = group.isin(a), group.isin(b)
#         tr, cv = [bigF.X.index[~ho][trcv] for trcv in [tr,cv]]
#         return ((tr, cv),(cv,tr))
    
#     def func(*, t):
#         ho = __class__.ho = bigF.Y.quarter>=2015.5; ho.name = None
#         t += LSM.t_offset
#         group = bigF.Y.quarter[~ho]
#         h = frozenset(group)
#         g = frozenset(random.Random(t).sample(h, len(h)//2))
#         return O(Samps=__class__.ctor2samps((g,h-g)), Ctor=(g,h-g))

    def tron2samp(cv_on):
        ho, group, index = __class__.ho, __class__.group, __class__.index
        tr, cv = ~group.isin(cv_on), group.isin(cv_on)
        if not cv.any():
            cv = tr
        tr, cv = [index[trcv] for trcv in [tr,cv]]
        return tr, cv

    def func(*, t):
        ho = __class__.ho = bigF.Y.quarter>=2015.5; ho.name = None
        __class__.group, __class__.index = bigF.Y.quarter[~ho], bigF.X.index[~ho]
        blocks = [[2010.,2010.25],[2010.5,2010.75],[2011.,2011.25],[2011.5,2011.75],[2012.,2012.25],[2012.5,2012.75],
                  [2013.,2013.25],[2013.5,2013.75],[2014.,2014.25],[2014.5,2014.75],[2015.,2015.25],[]]
        samples = [__class__.tron2samp(block) for block in blocks]
        return O(Samps=samples, Ctor=blocks)

@inside(LSM)
class Data(metaclass=staticclass):
    '''return the X Y data tables'''
    def func(*, t):
        return bigF.X, bigF.Y
    
@inside(LSM)
class Holdout(metaclass=staticclass):
    def func(*, t):
        return [bigF.Y.quarter.isin([2015.5,2015.75]),
                bigF.Y.quarter.isin([2016.,2016.25]),
                bigF.Y.quarter.isin([2016.5,2016.75])]

@inside(LSM, name='Params')
class Params(O()):
    '''parameters constant settings'''
    data = dict(
        objective = 'binary',
        #early_stopping_round = 32,
        metric = 'None',
        seed = 44,
        bagging_seed = 45,
        feature_fraction_seed = 46,
    )

# LIVE TESTING WALK ONLY

In [8]:
# @REAL
DISPLAY = True

TAG = 'ikg'
DIR = Path(top_dir/f'data/careful/');    DIR.mkdir(exist_ok=True)

LOG_DIR = DIR/'runs';    LOG_DIR.mkdir(exist_ok=True)
LOG = logger(file=LOG_DIR/(TAG+'.log'))
SEE = LOG
import given; given.SEE = SEE

VERBOSE_EVAL = 50

In [ ]:
#from search.LSM import LSM
mm = ModelManager(DIR)
ms = ModelSearch(specs=LSM, mm=mm, tag=TAG, log=LOG.logger('>>>', display=DISPLAY), verbose_eval=VERBOSE_EVAL)
walk = ms.walk()

In [ ]:
%%time
for (fi, si), results in walk:
    for a in ['nbest', 'train', 'scores', 'score', 'holdout']:
        SEE(f'{a} = {results[a]}')

>>> t = 0 ===================================================================================================
setup_training %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
len(feats) = 96
<InitKaggleGo>
len(samps) = 12
[[2010.0, 2010.25], [2010.5, 2010.75], [2011.0, 2011.25], [2011.5, 2011.75], [2012.0, 2012.25], [2012.5, 2012.75], [2013.0, 2013.25], [2013.5, 2013.75], [2014.0, 2014.25], [2014.5, 2014.75], [2015.0, 2015.25], []]
{'learning_rate': 0.05, 'max_depth': 8, 'num_leaves': 256, 'num_iterations': 1100}
{'min_data_in_leaf': 120, 'min_sum_hessian_in_leaf': 50, 'lambda_l1': 0, 'lambda_l2': 0}
>>>  k = 0 ---------------------------------------------------------------------------------------------------
>>> training...
[50]	tr's sharpe: 0.713028	tr's mean: 1.9752	tr's var: 7.67378	cv's sharpe: 0.607614	cv's mean: 1.08323	cv's var: 3.17823	ho0's sharpe: 0.579143	ho0's mean: 1.37577	ho0's var: 5.64312	ho1's sharpe: 0.64865	ho1's mean: 1.32218	ho1's var: 4.1549	ho2's sharpe: 0.306833	ho2

[150]	tr's sharpe: 0.782505	tr's mean: 2.72277	tr's var: 12.1073	cv's sharpe: 0.578052	cv's mean: 0.974199	cv's var: 2.84028	ho0's sharpe: 0.540537	ho0's mean: 1.5717	ho0's var: 8.45454	ho1's sharpe: 0.702046	ho1's mean: 1.61542	ho1's var: 5.29472	ho2's sharpe: 0.32584	ho2's mean: 0.756642	ho2's var: 5.39229
[200]	tr's sharpe: 0.810332	tr's mean: 2.96152	tr's var: 13.3569	cv's sharpe: 0.598849	cv's mean: 1.00581	cv's var: 2.82094	ho0's sharpe: 0.545394	ho0's mean: 1.61654	ho0's var: 8.78525	ho1's sharpe: 0.70786	ho1's mean: 1.65827	ho1's var: 5.48805	ho2's sharpe: 0.341338	ho2's mean: 0.813466	ho2's var: 5.67949
[250]	tr's sharpe: 0.832969	tr's mean: 3.16815	tr's var: 14.4662	cv's sharpe: 0.608345	cv's mean: 1.02435	cv's var: 2.83528	ho0's sharpe: 0.547977	ho0's mean: 1.63924	ho0's var: 8.94866	ho1's sharpe: 0.721091	ho1's mean: 1.70388	ho1's var: 5.58339	ho2's sharpe: 0.353531	ho2's mean: 0.859028	ho2's var: 5.90416
[300]	tr's sharpe: 0.853722	tr's mean: 3.35725	tr's var: 15.4645	cv's

[400]	tr's sharpe: 0.888516	tr's mean: 3.63757	tr's var: 16.7607	cv's sharpe: 0.815949	cv's mean: 1.49444	cv's var: 3.35455	ho0's sharpe: 0.553679	ho0's mean: 1.67607	ho0's var: 9.16358	ho1's sharpe: 0.731933	ho1's mean: 1.71993	ho1's var: 5.52179	ho2's sharpe: 0.404454	ho2's mean: 1.01442	ho2's var: 6.29071
[450]	tr's sharpe: 0.904265	tr's mean: 3.77827	tr's var: 17.458	cv's sharpe: 0.817306	cv's mean: 1.50807	cv's var: 3.40464	ho0's sharpe: 0.555098	ho0's mean: 1.6869	ho0's var: 9.23511	ho1's sharpe: 0.730523	ho1's mean: 1.73241	ho1's var: 5.62385	ho2's sharpe: 0.406345	ho2's mean: 1.03175	ho2's var: 6.44699
[500]	tr's sharpe: 0.914745	tr's mean: 3.91011	tr's var: 18.2716	cv's sharpe: 0.817136	cv's mean: 1.52172	cv's var: 3.46802	ho0's sharpe: 0.5513	ho0's mean: 1.68408	ho0's var: 9.3314	ho1's sharpe: 0.736973	ho1's mean: 1.75427	ho1's var: 5.66619	ho2's sharpe: 0.413459	ho2's mean: 1.06206	ho2's var: 6.59825
[550]	tr's sharpe: 0.928677	tr's mean: 4.04232	tr's var: 18.9466	cv's sharp

[650]	tr's sharpe: 0.928631	tr's mean: 4.23286	tr's var: 20.777	cv's sharpe: 0.473382	cv's mean: 1.54047	cv's var: 10.5896	ho0's sharpe: 0.517288	ho0's mean: 1.62092	ho0's var: 9.8188	ho1's sharpe: 0.800711	ho1's mean: 1.99694	ho1's var: 6.21985	ho2's sharpe: 0.420338	ho2's mean: 1.13425	ho2's var: 7.2815
[700]	tr's sharpe: 0.939105	tr's mean: 4.3526	tr's var: 21.4817	cv's sharpe: 0.469358	cv's mean: 1.53385	cv's var: 10.6797	ho0's sharpe: 0.517864	ho0's mean: 1.62532	ho0's var: 9.85025	ho1's sharpe: 0.800433	ho1's mean: 2.00829	ho1's var: 6.29513	ho2's sharpe: 0.42037	ho2's mean: 1.14749	ho2's var: 7.45137
[750]	tr's sharpe: 0.949087	tr's mean: 4.46091	tr's var: 22.0919	cv's sharpe: 0.466654	cv's mean: 1.53332	cv's var: 10.7963	ho0's sharpe: 0.518959	ho0's mean: 1.63458	ho0's var: 9.92075	ho1's sharpe: 0.794371	ho1's mean: 2.00639	ho1's var: 6.37949	ho2's sharpe: 0.422347	ho2's mean: 1.15826	ho2's var: 7.52096
[800]	tr's sharpe: 0.960921	tr's mean: 4.57178	tr's var: 22.6358	cv's sharp

[900]	tr's sharpe: 0.950775	tr's mean: 4.64067	tr's var: 23.8236	cv's sharpe: 1.05247	cv's mean: 2.32071	cv's var: 4.86206	ho0's sharpe: 0.508726	ho0's mean: 1.67019	ho0's var: 10.7786	ho1's sharpe: 0.747067	ho1's mean: 1.90147	ho1's var: 6.47829	ho2's sharpe: 0.436825	ho2's mean: 1.18576	ho2's var: 7.36847
[950]	tr's sharpe: 0.958425	tr's mean: 4.71708	tr's var: 24.2232	cv's sharpe: 1.05317	cv's mean: 2.32567	cv's var: 4.87641	ho0's sharpe: 0.506566	ho0's mean: 1.66653	ho0's var: 10.8232	ho1's sharpe: 0.742414	ho1's mean: 1.90281	ho1's var: 6.56903	ho2's sharpe: 0.436332	ho2's mean: 1.18743	ho2's var: 7.40602
[1000]	tr's sharpe: 0.961717	tr's mean: 4.78953	tr's var: 24.8023	cv's sharpe: 1.0553	cv's mean: 2.34	cv's var: 4.91681	ho0's sharpe: 0.503284	ho0's mean: 1.66004	ho0's var: 10.8795	ho1's sharpe: 0.740409	ho1's mean: 1.90275	ho1's var: 6.60418	ho2's sharpe: 0.441231	ho2's mean: 1.20363	ho2's var: 7.44137
[1050]	tr's sharpe: 0.970754	tr's mean: 4.89179	tr's var: 25.3931	cv's sharp

...

[450]	tr's sharpe: 2.32896	tr's mean: 27.5914	tr's var: 140.354	cv's sharpe: 0.624397	cv's mean: 1.84263	cv's var: 8.70871	ho0's sharpe: 0.534114	ho0's mean: 2.23867	ho0's var: 17.5677	ho1's sharpe: 0.555294	ho1's mean: 1.93517	ho1's var: 12.1449	ho2's sharpe: 0.359756	ho2's mean: 1.26182	ho2's var: 12.3022
CPU times: user 2h 1min 49s, sys: 37.7 s, total: 2h 2min 27s
Wall time: 15min 19s
[50]	tr's sharpe: 1.4991	tr's mean: 8.99279	tr's var: 35.9853	cv's sharpe: 0.845447	cv's mean: 1.3521	cv's var: 2.55767	ho0's sharpe: 0.540146	ho0's mean: 1.54633	ho0's var: 8.19563	ho1's sharpe: 0.662954	ho1's mean: 1.52069	ho1's var: 5.26154	ho2's sharpe: 0.387326	ho2's mean: 0.933159	ho2's var: 5.80442
[100]	tr's sharpe: 1.71633	tr's mean: 14.0904	tr's var: 67.3974	cv's sharpe: 0.844354	cv's mean: 1.53945	cv's var: 3.32416	ho0's sharpe: 0.527397	ho0's mean: 1.757	ho0's var: 11.0986	ho1's sharpe: 0.649773	ho1's mean: 1.77172	ho1's var: 7.43471	ho2's sharpe: 0.4026	ho2's mean: 1.12085	ho2's var: 7.750

[400]	tr's sharpe: 2.28683	tr's mean: 26.7637	tr's var: 136.97	cv's sharpe: 0.614516	cv's mean: 1.7199	cv's var: 7.83319	ho0's sharpe: 0.523719	ho0's mean: 2.11828	ho0's var: 16.3596	ho1's sharpe: 0.586921	ho1's mean: 1.93536	ho1's var: 10.8734	ho2's sharpe: 0.381968	ho2's mean: 1.30896	ho2's var: 11.7434
[450]	tr's sharpe: 2.32932	tr's mean: 27.6532	tr's var: 140.939	cv's sharpe: 0.615441	cv's mean: 1.76372	cv's var: 8.21273	ho0's sharpe: 0.521685	ho0's mean: 2.14129	ho0's var: 16.8475	ho1's sharpe: 0.5769	ho1's mean: 1.9402	ho1's var: 11.3108	ho2's sharpe: 0.381799	ho2's mean: 1.32964	ho2's var: 12.1283
CPU times: user 2h 2min 25s, sys: 35.5 s, total: 2h 3min 1s
Wall time: 15min 23s
[50]	tr's sharpe: 1.51835	tr's mean: 9.07141	tr's var: 35.6947	cv's sharpe: 0.659613	cv's mean: 1.52027	cv's var: 5.31205	ho0's sharpe: 0.538146	ho0's mean: 1.51837	ho0's var: 7.96077	ho1's sharpe: 0.699063	ho1's mean: 1.54019	ho1's var: 4.8542	ho2's sharpe: 0.381849	ho2's mean: 0.894795	ho2's var: 5.4911

[350]	tr's sharpe: 2.23278	tr's mean: 24.9336	tr's var: 124.704	cv's sharpe: 2.23278	cv's mean: 24.9336	cv's var: 124.704	ho0's sharpe: 0.539611	ho0's mean: 2.14671	ho0's var: 15.8265	ho1's sharpe: 0.631135	ho1's mean: 2.15337	ho1's var: 11.6411	ho2's sharpe: 0.408318	ho2's mean: 1.37975	ho2's var: 11.4184
[400]	tr's sharpe: 2.28826	tr's mean: 26.0474	tr's var: 129.574	cv's sharpe: 2.28826	cv's mean: 26.0474	cv's var: 129.574	ho0's sharpe: 0.540121	ho0's mean: 2.18341	ho0's var: 16.3414	ho1's sharpe: 0.628867	ho1's mean: 2.18314	ho1's var: 12.0517	ho2's sharpe: 0.413059	ho2's mean: 1.41801	ho2's var: 11.785
[450]	tr's sharpe: 2.33536	tr's mean: 26.9787	tr's var: 133.455	cv's sharpe: 2.33536	cv's mean: 26.9787	cv's var: 133.455	ho0's sharpe: 0.5432	ho0's mean: 2.23062	ho0's var: 16.8628	ho1's sharpe: 0.62688	ho1's mean: 2.20928	ho1's var: 12.4203	ho2's sharpe: 0.418558	ho2's mean: 1.45991	ho2's var: 12.1659
CPU times: user 2h 32min 14s, sys: 52.4 s, total: 2h 33min 7s
Wall time: 19min 9

CPU times: user 1h 2min 43s, sys: 22 s, total: 1h 3min 5s
Wall time: 7min 53s
[50]	tr's sharpe: 1.01348	tr's mean: 3.40404	tr's var: 11.2812	cv's sharpe: 0.803495	cv's mean: 1.33021	cv's var: 2.74078	ho0's sharpe: 0.55719	ho0's mean: 1.50843	ho0's var: 7.32901	ho1's sharpe: 0.724326	ho1's mean: 1.47217	ho1's var: 4.13093	ho2's sharpe: 0.370269	ho2's mean: 0.817062	ho2's var: 4.86939
[100]	tr's sharpe: 1.20163	tr's mean: 5.12774	tr's var: 18.21	cv's sharpe: 0.790573	cv's mean: 1.53196	cv's var: 3.75499	ho0's sharpe: 0.569214	ho0's mean: 1.75945	ho0's var: 9.55443	ho1's sharpe: 0.715181	ho1's mean: 1.71208	ho1's var: 5.73083	ho2's sharpe: 0.386206	ho2's mean: 1.01384	ho2's var: 6.89125
[150]	tr's sharpe: 1.33688	tr's mean: 6.51097	tr's var: 23.7196	cv's sharpe: 0.780937	cv's mean: 1.59014	cv's var: 4.14611	ho0's sharpe: 0.568814	ho0's mean: 1.83006	ho0's var: 10.3512	ho1's sharpe: 0.742998	ho1's mean: 1.86266	ho1's var: 6.28479	ho2's sharpe: 0.399712	ho2's mean: 1.11793	ho2's var: 7.8223

[450]	tr's sharpe: 1.67731	tr's mean: 11.2917	tr's var: 45.3203	cv's sharpe: 0.97494	cv's mean: 2.38711	cv's var: 5.995	ho0's sharpe: 0.494308	ho0's mean: 1.79576	ho0's var: 13.1977	ho1's sharpe: 0.756177	ho1's mean: 2.18478	ho1's var: 8.34768	ho2's sharpe: 0.452091	ho2's mean: 1.37376	ho2's var: 9.23354
CPU times: user 1h 2min 39s, sys: 20.7 s, total: 1h 3min
Wall time: 7min 53s
[50]	tr's sharpe: 1.02381	tr's mean: 3.42713	tr's var: 11.2054	cv's sharpe: 0.624099	cv's mean: 1.14176	cv's var: 3.3469	ho0's sharpe: 0.572475	ho0's mean: 1.54072	ho0's var: 7.2433	ho1's sharpe: 0.664519	ho1's mean: 1.3775	ho1's var: 4.29701	ho2's sharpe: 0.328829	ho2's mean: 0.726541	ho2's var: 4.88181
[100]	tr's sharpe: 1.20589	tr's mean: 5.11291	tr's var: 17.9772	cv's sharpe: 0.642958	cv's mean: 1.32737	cv's var: 4.26205	ho0's sharpe: 0.580757	ho0's mean: 1.78442	ho0's var: 9.44077	ho1's sharpe: 0.660194	ho1's mean: 1.56081	ho1's var: 5.58931	ho2's sharpe: 0.350752	ho2's mean: 0.916437	ho2's var: 6.82659
[

# see some stuff

In [13]:
ho = LSM.Samples.ho

In [14]:
import shap

# load JS visualization code to notebook
shap.initjs()

# train XGBoost model

# explain the model's predictions using SHAP values
# (same syntax works for LightGBM, CatBoost, and scikit-learn models)
explainer = shap.TreeExplainer(ms._gbm)
shap_values = explainer.shap_values(bigF.X[ho])

In [15]:
# visualize the first prediction's explanation
shap.force_plot(explainer.expected_value, shap_values[0,:], bigF.X[ho].iloc[0,:])